# Projet de permanence : Découverte des outils de classification et de clustering

## Prérequis

Un peu de curiosité et de patience feront l'affaire ! ;)

Plus sérieusement, il vous faut soit [Google colab'](https://colab.research.google.com/), soit un environnement Python avec `tensorflow`, `numpy`, `keras`et `scikit-learn`.

## Introduction

L'objectif de ce sujet est de vous faire découvrir les librairies **Keras** et **scikit-learn** en vous permettant d'avoir vos premiers résultats de **Machine Learning** avec le dataset **CIFAR10**.

Nous allons aborder les notions suivantes : les couches denses, les CNNs, les auto-encodeurs, le clustering, ...

Le plan est le suivant :

1. **La classification supervisée :** Prédiction du label d'une image grâce à un réseau de neurones
2. **Les autoencodeurs :** Concept et 3 applications qui vont changer votre vie !
3. **La classification non-supervisée :** Le secret derrière Argos.

Ci-dessous une petite FAQ pour vous présenter certains points. Si vous avez des questions ou des remarques, n'hésitez pas à les poser aux 2As présents en perm' ou sur le groupe Automatants. Good luck, have fun !

## FAQ

### Comment ça marche un fichier Jupyter ?

Les fichiers Jupyter vous permettent d'exécuter du Python depuis votre navigateur, tout en pouvant alterner morceaux de code et morceaux de textes. Afin d'éxécuter du code, il faut appuyer sur `Crtl` + `R` (Reste sur le bloc courant) ou `Shift` + `R` (Exécute le bloc courant et passe au bloc suivant).

Tous les `print` ou `plot` que vous ferez s'afficheront directement en dessus du bloc de code.

### Qu'est-ce qu'un dataset ?

Un **dataset** est un jeu de données, qui généralement associe à des données d'entrée une ou plusieurs données de sortie.

Par exemple, le dataset MNIST associe à des images en nuance de gris le chiffre qu'elle représente.

Ici, nous allons manipuler le dataset CIFAR10 qui à des images en couleur associe le nom de de ce qu'elle représente (Plus de détails dans la suite).

La construction d'un dataset bien labelisé et conséquent est une des difficultés majeures en Machine Learning. Heureusement pour nous, de nombreux datasets existent et sont mis à disposition de tous !

### Mais c'est quoi Keras?

**Keras** est une libraire qui permet de rapidement créer des réseaux de neurones et de les entrainer. De nombreuses couches sont déjà codées, telles que les couches denses (du multi-perceptron), des CNNs, des couches récurrentes, etc., ce qui permet de n'avoir qu'à les assembler. Keras inclut également plusieurs datasets, ce qui est très pratique pour tester des algorithmes.

### Et scikit-learn ?

De son côté **scikit-learn** propose beaucoup d'outils pour faire de l'analyse prédictive de données (Classification, régression, clustering, réduction de dimension, etc. *Beaucoup de termes barbares que vous allez découvrir dans la suite*).

# Import des librairies et des données

## Les librairies

Commençons par importer les librairies dont nous aurons besoin.

In [ ]:
from time import time
import tensorflow as tf
import numpy as np

from tensorflow import keras

from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import matplotlib

## Le dataset

Importons maintenant le dataset. Nous allons utiliser CIFAR10, dont voilà une description :

> L'ensemble de données CIFAR10 contient 60 000 images couleur de 32x32 dans 10 classes, avec 6 000 images dans chaque classe. L'ensemble de données est divisé en 50 000 images d'entraînement et 10 000 images de test. Les classes sont mutuellement exclusives et il n'y a pas de chevauchement entre elles.

Pour rappel, il est important de séparer les données en données d'entrainement et données de test. Cela permet de constater si notre modèle a effectué un sur-apprentissage (overfitting) et est incapable de généraliser ce qu'il a appris.

**Keras** inclut de nombreux datasets dans son modèle `datasets` ([liste des datasets](https://keras.io/api/datasets/)) que l'on peut importer avec la fonction `load_data`.

Les valeurs des pixels varient entre 0 et 255. Nous allons les normaliser pour qu'elles soient entre 0 et 1, ce qui est plus adapté pour les réseaux de neurones.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

Visualisons un peu toutes ces images !

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

Parfait, nous pouvons maintenant commencer les choses sérieuses !

# Classification supervisée

## Qu'est-ce que la classification supervisée ?

L'idée est de trouver une fonction de prédiction à partir des données annotées. Ici, cela revient à trouver la fonction qui à chaque image associe son label.

Cela est différent de l'apprentissage non-supervisé que l'on fera dans la suite.

## Comment allons-nous faire ?

Les images font 32 pixels par 32 avec 3 valeurs pour chaque couleur (**R**ouge, **V**ert, **B**leu ou **RVB** en français et **RGB** en anglais). Nous avons donc 32 x 32 x 3 = 3072 paramètres ...

Nous allons donc utiliser des réseaux de neurones pour approximer notre fonction de prédiction !

## Avec un réseau dense

### Rappel théorique

Si vous assistez aux premières formations, vous savez ce qu'est un réseau dense et si vous avez assisté au TP Multiperceptron, vous avez même codé votre propre réseau dense !

Pour ceux qui ne s'en souvienne plus, un réseau de neurones dense est composée de plusieurs couches, elles-mêmes composées de plusieurs noeuds. Chaque noeud est relié à tous les noeuds de la couche précédente, dont il pondère la somme des valeurs par des poids, et décale la valeur obtenue par un biais. Finalement, pour éviter que l'ensemble de toutes les couches se résument à une seule couche, on casse la linéarité en associant à chaque couche une fonction d'activation (sigmoïde, tanh, relu, etc.).

C'est une explication très très succinte. Pour mieux comprendre ce que c'est, vous pouvez revoir la formation sur les premières formations, ou il y a ces excellentes vidéos de [Science4All, Les réseaux de neurones | Intelligence artificielle 41](https://www.youtube.com/watch?v=8qL2lSQd9L8) (FR) et de [3Blue1Brown, But what is a Neural Network?](https://www.youtube.com/watch?v=aircAruvnKk) (ANG, mais très visuel).

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/Neural_network.svg/220px-Neural_network.svg.png)

La première couche est appelée `Input layer`, car c'est là que sont insérées les données d'entrée. La dernière couche est appelée `Output layer` et correspond aux valeurs de sortie. Les couches intermédiaires sont appelées `Hidden layer`.

### En pratique

Pour créer un modèle avec **Keras**, on peut utiliser `keras.models.Sequential()`.

**Instruction :** Créer un modèle que vous assignerez à `model_dense1`.

In [ ]:
model_dense1 = keras.models.Sequential()

Il est maintenant temps d'ajouter les premières couches à ce modèle. On pourra utiliser :
- `model_dense1.add(layer)` qui ajoute layer à la suite des couches du modèle.


- `keras.layers.Dense(n_noeuds, activation='...')` qui renvoie une couche dense avec `n_noeuds` noeuds et comme fonction d'activation celle que vous avez précisez (les options possibles sont notamment `sigmoid`, `tanh`, `relu`, `softmax`)

  Plus d'infos dans la doc de Keras : [Dense Layer](https://keras.io/api/layers/core_layers/dense/).


- `keras.layers.Flatten(input_shape=shape)` qui renvoie une couche qui *aplatit* les données d'entrée en une liste à la sortie.

  `input_shape` est un tuple qui permet de préciser le format des données d'entrée (Exemple (32, 32, 3) pour les images de CIFAR10).

Maintenant, réfléchissons un peu. Nos données d'entrée sont de la forme (32, 32, 3). Or, une couche dense ne peut prendre qu'une liste en entrée ... Il va donc falloir utiliser utiliser une première couche `Flatten` pour avoir le bon format.

**Instruction :** Rajoutez une couche `Flatten` au modèle.

*Note :* On pourrait également modifier les données d'entrée, mais vu que dans la suite, on va remanier les images, ça ne serait pas pratique.

In [ ]:
model_dense1.add(keras.layers.Flatten(input_shape=(32, 32, 3)))

À la sortie de cette couche, nous avons donc une liste de 32 x 32 x 3 valeurs. On peut donc enfin mettre une première couche cachée !

**Instruction :** Rajoutez une couche `Dense` avec 512 noeuds et la fonction d'activation `relu`.

In [ ]:
model_dense1.add(keras.layers.Dense(512, activation='relu'))

Ajoutons maintenant la couche de sortie, on a 10 classes possibles. Notre dernière couche aura donc 10 noeuds, et on aimerait que notre réseau prédise la probabilité de chaque classe, on utilisera donc la fonction d'activation `softmax`.

**Instruction :** Rajoutez une couche `Dense` avec 10 noeuds et la fonction d'activation `softmax`.

In [ ]:
model_dense1.add(keras.layers.Dense(10, activation='softmax'))

#### Visualisation du modèle, et compilation

Vous avez créé votre premier modèle, félicitations ! Voyons voir un peu ce qu'il contient.

La fonction `summary()` affiche un résumé des couches de votre modèle, ainsi que le nombres de paramètres entrainables, etc.

In [ ]:
model_dense1.summary()

Avant d'entrainer notre modèle, il faut qu'on le compile pour l'entrainement. Cela permet notamment de préciser certains paramètres.

On utilisera la fonction `compile(optimizer="adam", loss=..., metrics=...)` :
- `optimizer`
  
  Méthode utilisée lors de la backpropagation. Ici, on utilisera Adam, qui est beaucoup une méthode d'optimisation très efficace, plus efficace que la descente de gradient normale que vous avez vu en formation.

  Plus d'infos sur les optimisateurs : [Various Optimization Algorithms For Training Neural Network](https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6).
  
  
- `loss`

  Précise comment on calcule l'erreur. On utilisera `tf.keras.losses.CategoricalCrossentropy(from_logits=True)` ici.
  
- `metrics`

  Liste des métriques qu'on aimerait suivre. Celle la précision nous intéresse ici, on prendra donc `['accuracy']`.
  
**Instruction :** Compiler votre modèle avec `model_dense1.compile(...)`.

In [ ]:
model_dense1.compile(optimizer='adam',
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                     metrics=['accuracy'])

#### Entrainement

Notre modèle est fin prêt pour être entrainé !

On utilisera :

`history = model_dense1.fit(x, y, epochs=..., validation_data=(x_test, y_test))`

- `x`
  
  Les données d'entrée, ici `train_images`.
  
  
- `y`
  
  Les données de sortie, ici `train_labels`.
  
  
- `epochs`
  
  Nombre d'époques à faire pour entrainer notre modèle. Une époque correspond à un passage sur l'ensemble des données `x` et `y`.
  
  On pourra prendra 5 ici.
  
  
- `validation_data=(x_test, y_test)`

  Couple de données entrée / sortie qui ne sont pas utilisées pour entrainer le modèle. Une validation est faite à chaque fin d'époque, et cela permet de voir si notre réseau réussit à généraliser à des nouvelles données ou non.
  
  Nos données de validation sont `test_images` et `test_labels` ici.
  
La fonction `fit()` renvoie l'historique des métriques au cours de l'entrainement.

**Instruction :** Entrainer votre modèle avec `model_dense1.fit(...)`.

In [ ]:
history = model_dense1.fit(train_images, train_labels,
                           epochs=2,
                           validation_data=(test_images, test_labels))

Durant l'entrainement, vous voyez pour chaque époque : l'avancement, le temps restant estimé, la loss et la précision. Une fois que l'époque est finie, vous voyez également val_loss et val_accuracy, qui sont la loss et la précision sur les données de test.

Affichons les données de l'historique, et voyons comment notre modèle s'en sort !

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

On constate qu'on a une précision d'environ 45%, ce qui est mieux que le hasard, mais n'est pas exceptionnel non plus.

Vous pouvez vous amuser à recréer un modèle en rajoutant plus de couches avec des paramètres différents, et voir si vous arrivez à atteindre une meilleure précision ! ;)

## Avec des CNNs

Jusqu'ici, nous avons utilisé des couches denses pour notre réseau de neurones, mais cela n'est pas le plus adapté pour traiter des images. En effet, cela n'est pas insensible aux translations, etc. C'est pourquoi nous allons utiliser des CNN (Convolutional Neural Network) dans la suite.

### Un peu de théorie

Les CNNs sont spécifiquement faits pour travailler sur des images !

Chaque image est un tableau de taille *largeur x longueur x canaux* (les canaux peuvent être au nombre de 3 pour une image couleur ou 1 pour une image en nuances de gris. On applique alors un filtre, qui est un tableau de largeur et longueur inférieures, mais de nombres de canaux identiques ! Le filtre sert de coefficients pour sommer la valeur des pixels en dessous, et on décale le filtre jusqu'à avoir couvert toute l'image.

Ci-dessus un exemple.

![](https://miro.medium.com/max/413/1*4yv0yIH0nVhSOv3AkLUIiw.png)
![](https://miro.medium.com/max/268/1*MrGSULUtkXc0Ou07QouV8A.gif)

Voilà, quelques filtres communs, on voit que cela peut servir pour la détection de bord par exemple !

![](https://miro.medium.com/max/349/1*uJpkfkm2Lr72mJtRaqoKZg.png)

Il y a d'autres paramètres qu'on ne détaillera pas ici. Au sein d'une couche CNN, il peut y avoir plusieurs filtres. La sortie est alors une image de même largeur et longueur, mais le nombre de canaux est alors égale au nombre de filtres. L'entrainement d'un CNN revient à déterminer les coefficients de ces filtres.

Cependant, vous remarquerez qu'ici, on ne diminue pas la taille d'une image, mais qu'on joue seulement sur la profondeur de celle-ci. Afin, de réduire la dimension de celle-ci, nous allons utiliser du `Pooling`, notamment, du `MaxPooling` qui revient à considérer la valeur maximale en déplaçant un filtre à chaque fois (Voir l'illustration ci-dessous).

![](https://miro.medium.com/max/602/1*SmiydxM5lbTjoKWYPiuzWQ.png)

La `MaxPooling` réduit donc la surface de l'image, tout en gardant la même profondeur. Par exemple, avoir un carré de *2x2* pour le `MaxPooling` divisera la largeur et la longeur de l'image sortante par 2.

Le fait de chainer des CNNs permet au modèle d'apprendre des motifs complexes. Les premiers CNNs détecteront les bords, etc. Tandis que les suivants pourront agréger tout ça pour reconnaitre des formes plus évoluées. Après les CNNs, nous pouvons mettre un réseau dense, afin de faire la classification à partir des *features* extraites des CNNs.

### En pratique

Avec **Keras**, on peut définir un CNN avec les couches suivantes :

- `keras.layers.Conv2D(n_filtres, longueur_filtre, activation='relu')`

  - `n_filtres` : Le nombre de filtres du CNN
  
  - `longueur_filtre` : Définit la largeur du filtre qui sera un carré.
  
  - `activation` : Fonction d'activation, on prendra `'relu'`
  
  
- `keras.layers.MaxPooling(largeur)`

  - `largeur` : Largeur du carré du Pooling
  
Dans la partie CNN du modèle, on alternera les couches `Conv2D` et `MaxPooling`.
  
Dans la définition de notre modèle, puisque les CNNs prennent en entrée une image, nous n'avons plus besoin de la couche `Flatten` au début.

#### Définition du modèle

**Instruction :**
1. Créer un modèle que vous assignerez à `model_cnn1`.
2. Rajoutez une couche `Conv2D` avec 64 filtres, une largeur de 3, la fonction d'activation relu et input_shape=(32, 32, 3) (Il faut préciser pour la première couche le format des données d'entrée).
3. Rajoutez une couche `MaxPooling` de largeur 3.
2. Rajoutez une couche `Conv2D` avec 64 filtres, une largeur de 3 et la fonction d'activation relu.
3. Rajoutez une couche `MaxPooling` de largeur 3.
5. Rajoutez une couche `Conv2D` avec 64 filtres, une largeur de 3 et la fonction d'activation relu.

Cela correspondra à notre partie CNN.

In [ ]:
model_cnn1 = models.Sequential()
model_cnn1.add(layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)))
model_cnn1.add(layers.MaxPooling2D(2))
model_cnn1.add(layers.Conv2D(64, 3, activation='relu'))
model_cnn1.add(layers.MaxPooling2D(2))
model_cnn1.add(layers.Conv2D(64, 3, activation='relu'))

Passons maintenant à la partie dense. À la dernière couche, les données sont sous la forme d'une tableau et non d'une liste. Il faut donc de nouveau utiliser la couche `Flatten` comme toute à l'heure!

**Instruction :**
1. Ajoutez une couche `Flatten`.
2. Rajoutez une couche `Dense` avec 64 noeuds et la fonction d'activation `relu`.
2. Rajoutez une couche `Dense` avec 10 noeuds et la fonction d'activation `softmax`.

In [ ]:
model_cnn1.add(layers.Flatten())
model_cnn1.add(layers.Dense(64, activation='relu'))
model_cnn1.add(layers.Dense(10, activation='softmax'))

#### Préparation du modèle à l'entrainement

**Instructions :**
1. Visualisez votre modèle avec `summary()`
2. Compiler votre modèle avec la fonction `compile()` et les mêmes paramètres que la partie prédédente.

In [ ]:
model_cnn1.summary()
model_cnn1.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [ ]:
history = model_cnn1.fit(train_images, train_labels,
                         epochs=2,
                         validation_data=(test_images, test_labels))

De la même façon que tout à l'heure, regardons les performances de notre modèle.

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

## Auto-encodeurs et applications diverses

Dans la partie précédente, on devinait directement à partir de l'image qu'elle était sa classe. Maintenant, nous allons supposer que nous ne connaissons pas les classes des images, et nous allons essayer de trouver des similarités entre toutes les images. Pour se faire, nous allons utiliser des auto-encodeurs.


## Un auto-enco-quoi ?

Les **auto-encodeurs** sont une famille de réseaux de neurones particuliers qui ont pour premier objectif de réduire la dimension de l'espace des données qui nous intéressent. Il se décomposent en deux parties :

*   Un encodeur qui doit apprendre la représentation des données d'entrée qui permet de passer de la dimension initiale à la dimension réduite.
*   Un décodeur qui doit reproduire l'entrée le plus fidèlement possible à partir de la représentation que donne l'encodeur.

On retrouve donc un structure caractéristique en "goulot d'étranglement" où les couches aux extrémités sont de la même dimension que la taille des données à caractériser et où la couche centrale est de la dimension de réduction souhaitée qui contiendra la représentation des entrées.

![](https://www.researchgate.net/publication/318204554/figure/fig1/AS:512595149770752@1499223615487/Autoencoder-architecture.png)

Pour entrainer ce réseau, on va seulement utilisée les données d'entrées : pas besoin donc de labéliser les données, on laisse la descente de gradient faire tout le travail : il s'agit d'un apprentissage non-supervisé.

## Un premier exemple

On a vu précédemment que les couches denses n'étaient pas très adaptées pour manipuler les images. Mais pour bien comprendre le concept de l'auto-encodeur, on va commencer avec ça.

Si vous avez bien suivi, un auto-encodeur est composé de deux modèles : un encodeur, et un décodeur. Ce dernier est souvent un symétrique de l'encodeur.

On va donc créer une classe `AutoencoderDense` qui contiendra ces deux modèles.

*Note :* Pas d'inquiétude si vous n'êtes pas familié avec les classes, vous en manipulez déjà depuis toute à l'heure sans le savoir, et ici la structure sera déjà présente, vous n'aurez qu'à compléter les parties manquantes.

`latent_dim` est la dimension de la couche latente de l'auto-encodeur, c'est-à-dire la couche au milieu de celui-ci.

**Instructions (Encodeur) :**
1. Rajoutez une couche `Flatten` en précisant `input_shape` à `encoder`. Les données d'entrée sont les images.
2. Rajoutez une couche `Dense` avec 1024 noeuds et la fonction d'activation `relu` à `encoder`.
3. Rajoutez une couche `Dense` avec `latent_dim` noeuds et la fonction d'activation `relu` à `encoder`.


**Instructions (Décodeur) :**
1. Rajoutez une couche `Dense` avec 1024 noeuds, la fonction d'activation `relu` et `input_shape` à `decoder`.

   Les données d'entrées sont les valeurs de la couche latente, qui est de dimension `latent_dim`.
   
   **Attention :** `input_shape` prend en argument un tuple, donc méfiez-vous ... Affichez (45) et (45,) si vous ne voyez pas le problème.


2. Rajoutez une couche `Dense` avec 32 * 32 * 3 noeuds (ie le nombre de valeurs d'entrée) et la fonction d'activation `sigmoid` à `decoder`.

   À votre avis, pourquoi utilise-t-on la fonction d'activation sigmoid plutôt que relu pour cette dernière couche ? Je vous invite à regarder sur Google à quoi ressemble ces deux fonctions, et à réfléchir à ce qu'on prévoit avec cette dernière couche.
   
   
3. Rajoutez une couche `Reshape((32, 32, 3))` à `decoder` afin de mettre tout ça sous la forme d'une image couleur.

In [ ]:
class AutoencoderDense(Model):
    def __init__(self, latent_dim):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        # Définition de l'encodeur
        encoder = keras.Sequential(name='encoder')
        encoder.add(layers.Flatten(input_shape=(32, 32, 3)))
        encoder.add(layers.Dense(1024, activation='relu'))
        encoder.add(layers.Dense(latent_dim, activation='relu')) # Couche latente
        
        # Définition du décodeur
        decoder = keras.Sequential(name='decoder')
        decoder.add(layers.Dense(1024, activation='relu', input_shape=(latent_dim,)))
        decoder.add(layers.Dense(32 * 32 * 3, activation='sigmoid'))
        decoder.add(layers.Reshape((32, 32, 3)))
        
        # Sauvegarde de l'encodeur et du décodeur
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def summary(self):
        self.encoder.summary()
        self.decoder.summary()


Vous venez de définir la classe de l'auto-encodeur, il ne reste plus qu'à en créer un !

**Instructions :**
1. Créez un autoencodeur avec : `autoencoder_dense = AutoencoderDense(latent_dim)`.


2. Affichez sa description avec : `autoencoder_dense.summary()`.

In [ ]:
latent_dim = 512
autoencoder_dense = AutoencoderDense(latent_dim)
autoencoder_dense.summary()

Pour ceux qui ne sont pas familier avec les classes, vous remarquez que vous venez de créer votre première classe ! ;)

Il ne reste plus qu'à le compiler, et à l'entrainer !

**Instructions :**
1. Compilez `autoencoder_dense` avec la fonction `compile` et les paramètres suivants : `'adam'` comme optimiseur et `losses.MeanSquaredError()` comme loss.


2. Entrainez `autoencoder_dense` avec la fonction `fit`.

   **Attention**, les données de sortie ne sont plus les mêmes ! En effet, on cherche à reconstruire l'image en entrée après avoir réduit sa dimension.

In [ ]:
autoencoder_dense.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_dense.fit(train_images, train_images,
                    epochs=1,
                    validation_data=(test_images, test_images))

Vous disposez maintenant d'un modèle capable, en théorie, de copier les images qu'on lui passe en entrée ... Regardons un peu ça.

Le code qui suit prend `n` indices au hasard, et affiche l'image originale et l'image en sortie de l'auto-encodeur correspondant à chaque indice.

Vous pouvez voir qu'on accède à l'encodeur et au décodeur de notre modèle à l'aide de `autoencoder_dense.encoder` et `autoencoder_dense.decoder`.

In [ ]:
n = 10
indices = np.random.choice(len(test_images), n)

encoded_imgs = autoencoder_dense.encoder(test_images[indices]).numpy()
decoded_imgs = autoencoder_dense.decoder(encoded_imgs).numpy()

plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_images[indices][i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Bilan ? C'est un peu flou tout ça. On pouvait s'y attendre. Notre réseau est composé uniquement de couches denses et le dataset est assez compliqué, notre modèle est donc très mal adapté.

C'est pourquoi nous allons utilisé des CNNs ! Après la partie précédente, vous avez au moins une vague idée de ce que c'est. L'essentiel à retenir est que les CNNs sont beaucoup plus efficaces pour traiter des images.

## Un deuxième exemple

De la même façon que précédemment, nous allons créer une classe `AutoencoderCNN`.

**Instructions (Encodeur) :**
1. Rajoutez une couche `Conv2D` de 64 filtres, de taille 3, de fonction d'activation `'relu'` et de `padding='same'` en précisant `input_shape` à `encoder`. 
2. Rajoutez une couche `MaxPooling2D` de taille 2 à `encoder`.
3. Rajoutez une couche `Conv2D` de 32 filtres, de taille 3, de fonction d'activation `'relu'` et de `padding='same'` à `encoder`. 
4. Rajoutez une couche `MaxPooling2D` de taille 2 à `encoder`.
3. Rajoutez une couche `Flatten` à `encoder`.
3. Rajoutez une couche `Dense` avec `latent_dim` noeuds et la fonction d'activation `relu` à `encoder`.


**Instructions (Décodeur) :**
1. Rajoutez une couche `Reshape((8, 8, 16))` en précisant `input_shape` à `decoder`. 
2. Rajoutez une couche `UpSampling2D` de taille 2 à `decoder`. Cette couche permet de doubler la longueur et largueur de l'image. Ce qui à un effet un peu "inverse" au MaxPooling.
3. Rajoutez une couche `Conv2D` de 16 filtres, de taille 3, de fonction d'activation `'relu'` et de `padding='same'` à `decoder`. 
4. Rajoutez une couche `UpSampling2D` de taille 2 à `decoder`.
5. Rajoutez une couche `Conv2D` de 32 filtres, de taille 3, de fonction d'activation `'relu'` et de `padding='same'` à `decoder`. 
6. Rajoutez une couche `Conv2D` de 3 filtres, de taille 3, de fonction d'activation `'sigmoid'` et de `padding='same'` à `decoder`. 

In [ ]:
latent_dim = 8 * 8 * 16

class AutoencoderCNN(Model):
    def __init__(self, latent_dim):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        # Définition de l'encodeur
        encoder = keras.Sequential(name='encoder')
        encoder.add(layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)))
        encoder.add(layers.MaxPooling2D(2))
        encoder.add(layers.Conv2D(16, 3, activation='relu', padding='same'))
        encoder.add(layers.MaxPooling2D(2))
        
        encoder.add(layers.Flatten()) # Préparation couche latente
        encoder.add(layers.Dense(latent_dim, activation='relu')) # Couche latente
        
        # Définition du décodeur
        decoder = keras.Sequential(name='decoder')
        decoder.add(layers.Reshape((8, 8, 16), input_shape=(latent_dim,)))
        decoder.add(layers.UpSampling2D(2))
        decoder.add(layers.Conv2D(16, 3, activation='relu', padding='same'))
        decoder.add(layers.UpSampling2D(2))
        decoder.add(layers.Conv2D(32, 3, activation='relu', padding='same'))
        decoder.add(layers.Conv2D(3, 3, activation='sigmoid', padding='same'))
        
        # Sauvegarde de l'encodeur et du décodeur
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def summary(self):
        self.encoder.summary()
        self.decoder.summary()


Il ne reste plus qu'à le créer, le compiler et l'entrainer maintenant.

**Instructions :**
1. Créez un autoencodeur avec : `autoencoder_cnn = AutoencoderCNN(latent_dim)`.
2. Compilez `autoencoder_cnn` avec a fonction `compile` et les paramètres suivants : `'adam'` comme optimiseur et `losses.MeanSquaredError()` comme loss.


3. Entrainez `autoencoder_cnn` avec la fonction `fit`.

In [ ]:
autoencoder_cnn = AutoencoderCNN(latent_dim)
autoencoder_cnn.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder_cnn.fit(train_images, train_images,
                    epochs=1,
                    validation_data=(test_images, test_images))

Regardons maintenant ce que le modèle réussit à faire !

In [ ]:
n = 10
indices = np.random.choice(len(test_images), n)

encoded_imgs = autoencoder_cnn.encoder(test_images[indices]).numpy()
decoded_imgs = autoencoder_cnn.decoder(encoded_imgs).numpy()

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_images[indices][i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

C'est mieux, mais ce n'est pas parfait encore ...

Il existe de nombreuses techniques pour améliorer les auto-encodeurs, mais nous n'allons pas nous intéresser à ça dans la suite du TP. Nous allons voir d'autres applications des auto-encodeurs, et d'autres techniques d'apprentissage non-supervisé.

## Applications diverses d'un auto-encodeur

Les auto-encodeurs peuvent servir pour la compression (avec perte), mais pas seulement !

On peut aussi s'en servir pour débruiter des images ou faire de la génération de contenu.

On va commencez par définir un auto-encodeur un peu plus efficace.

In [ ]:
class AutoencoderCNN2(Model):
    def __init__(self):
        super().__init__()
        
        # Définition de l'encodeur
        encoder = keras.Sequential([
            layers.Conv2D(32, 3, 1, activation='relu', padding='same'), # 32 x 32 x 32
            layers.BatchNormalization(),
            layers.Conv2D(32, 3, 2, activation='relu', padding='same'), # 16 x 16 x 32
            layers.Conv2D(32, 3, 1, activation='relu', padding='same'), # 16 x 16 x 32
            layers.BatchNormalization(),
            #layers.Conv2D(32, 3, 2, activation='relu', padding='same'), # 8 x 8 x 32
            #layers.Conv2D(32, 3, 1, activation='relu', padding='same'), # 8 x 8 x 32
        ], name='encoder')
        
        # Définition du décodeur
        decoder = keras.Sequential([
            #layers.UpSampling2D(), # 16 x 16 x 32
            #layers.Conv2D(32, 3, 1, activation='relu', padding='same'), # 16 x 16 x 32
            layers.UpSampling2D(), # 32 x 32 x 32
            layers.Conv2D(32, 3, 1, activation='relu', padding='same'), # 32 x 32 x 32
            layers.BatchNormalization(),
            layers.Conv2D(3, 1, 1, activation='sigmoid', padding='same') # 32 x 32 x 3
        ], name='decoder')
        
        # Sauvegarde de l'encodeur et du décodeur
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def summary(self):
        self.encoder.summary()
        self.decoder.summary()

### Le débruitage

On va entrainer notre modèle a prédire les images non-bruités à partir d'images bruitées.

In [ ]:
ae = AutoencoderCNN2()
ae.compile(optimizer='adam', loss=losses.MeanSquaredError())

À chaque époque, nous générons de nouvelles images bruitées.

In [ ]:
n_epochs = 1
NOISE = 0.1     #  Set to 0 for a regular (non-denoising...) autoencoder
for i in range(n_epochs):
    noise = np.random.normal(0, NOISE, train_images.shape)
    
    # Keep value between 0 and 1
    imgs = train_images + noise
    imgs = np.where(imgs > 1., 1., imgs)
    imgs = np.where(imgs < 0., 0., imgs)
    ae.fit(imgs, train_images, epochs=1)

Voyons voir ce que ça donne.

In [ ]:
n = 10
indices = np.random.choice(len(test_images), n)

NOISE = 0.1
imgs = (test_images + np.random.normal(0, NOISE, test_images.shape))[indices]

# Keep value between 0 and 1
imgs = np.where(imgs > 1., 1., imgs)
imgs = np.where(imgs < 0., 0., imgs)

denoised_imgs = ae(imgs)

plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(imgs[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(denoised_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

On voit que cela fonctionne plutôt bien et cela a des applications pratiques dans la vraie vie.

### La génération d'image

L'idée est la suivante : La couche latente est un vecteur de taille N que le décodeur sait transformer en image, donc si on peut générer des vecteurs aléatoires, et obtenir des nouvelles images, non ?

Dans un premier temps, on va déjà récupérer toutes les images avec des bateaux pour savoir exactement quelle classe le modèle essayera de générer.

In [ ]:
ind_train = train_labels == 8
ind_train = ind_train.reshape((-1,))
boats_imgs_train = train_images[ind_train, :, :]

ind_test = test_labels == 8
ind_test = ind_test.reshape((-1,))
boats_imgs_test = test_images[ind_test, :, :]

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(boats_imgs_train[i], cmap=plt.cm.binary)
plt.show()

Puis, nous allons entrainer un auto-encodeur sur ces magnifiques images de bateaux. Le nombre d'images étant divisé par 10, l'entrainement va allez beaucoup plus vite !

In [ ]:
ae_gen = AutoencoderCNN2()
ae_gen.compile(optimizer='adam', loss=losses.MeanSquaredError())

ae_gen.fit(boats_imgs_train, boats_imgs_train,
                    epochs=5,
                    validation_data=(boats_imgs_test, boats_imgs_test))

Essayons maintenant de générer des vecteurs aléatoires de la taille de la couche latente, et voyons si ça fonctionne ...

In [ ]:
random_vectors = np.random.normal(0, .3, size=(10, 16*16*32))
random_vectors = random_vectors.reshape((-1, 16, 16, 32))
generated_imgs = ae_gen.decoder(random_vectors)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(generated_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Bof ... C'est pas très concluant comme test, et c'est normal. En réalité, les 16 x 16 x 32 = 8192 coordonées du vecteur de l'espace latent sont corrélées, on ne peut donc pas mettre des valeurs complétement aléatoires. On va utiliser un algorithme linéaire afin de décorreler les variables de notre espace latent afin de s'assurer de générer des bateaux qui sont "dans la moyenne"; il s'agit de l'algorithme PCA (analyse en composante principale, pour les matheux c'est pas très compliqué et on peut l'implémenter soit même, voir sur [wikipédia](https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales)).

L'idée général est de trouver les axes qui expliquent le plus notre nuage de points. Bref, si c'est pas clair, vous le verrez durant le cours de Statistiques & Apprentissages.

Ici, on va utiliser le PCA déjà écrit de la librairie **scikit-learn**.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Génération de notre nuage de points
samples_pca = ae_gen.encoder(boats_imgs_test)
reshaped_samples_pca = tf.reshape(samples_pca, (-1, 16*16*32)) # On les reshape, car ils sont de la forme 16x16x32

N = 85
pca = PCA(n_components=N, svd_solver='full')
pca.fit(reshaped_samples_pca)

**Explication du code ci-dessus :**
- Dans un premier temps, on crée notre nuage de points, c'est-à-dire l'ensemble des vecteurs latents issus des images de test. 

- On crée un objet `PCA`. `n_components` précise le nombre d'axes qui nous intéresse, l'algorithme PCA effectura alors une réduction de dimension sur ces axes.

- Finalement, on ajuste les paramètres à partir de notre nuage de points avec la fonction `fit`.

Passons enfin à la génération de 49 magnifiques bateaux !

In [ ]:
n = 49

# On regarde comment varie les vecteurs latents réduits pour ne pas générer des valeurs trop différentes.
a = pca.transform(reshaped_samples_pca)
random_vectors = np.random.normal(loc=np.mean(a, axis=0), scale=np.std(a, axis=0), size=(n, N))

# On transforme les vecteurs en vecteurs valables pour l'espace latent
random_vectors = pca.inverse_transform(random_vectors) # Transforme les vecteurs réduits dans l'espace initial
random_vectors = random_vectors.reshape((-1, 16, 16, 32))

# On transforme les vecteurs de l'espace latent en image
decoded_random_images = ae_gen.decoder(random_vectors)

plt.figure(figsize=(15,15))
for i in range(49):
    plt.subplot(7,7,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(decoded_random_images[i], cmap=plt.cm.binary)
plt.show()

J'ai peut-être menti ... Ils ne sont pas si magnifiques, mais c'est déjà un début. Le type d'auto-encodeur que nous avons défini n'est pas vraiment fait pour la génération, les auto-encodeurs variationnels sont plus adaptés.

Je vous invite à jouer avec la variable `N` qui définit le paramètre `n_components` dans le code ci-dessus et de voir l'impact qu'il a sur les images générées.

# - Passer le truc en "mode TP" avec des instructions.
# - Problème dans le niveau des titres je crois


## Analyse des caractéristiques profondes de l'auto-encodeur et clustering

**Dans cette partie vous serez amenés à manipuler des méthodes et concepts que vous n'avez peut-être jamais vus et que nous n'avons pas encore expliqués dans les formations. Ce TP va vous permettre de les utiliser mais il n'y aura que très peu d'explications théoriques. On vous invite à lire les références que l'on vous donne pour comprendre le fonctionnement général des algorithmes. La plupart de ces méthodes vous seront présentées d'un point de vue plus théorique plus tard.**

###Vecteurs latents

Les vecteurs que l'on obtient après compression par l'auto-encodeur sont appelés **vecteurs latents**, ou **caractéristiques profondes**, ou **deep features**.
Ces termes ne sont pas réservés aux auto-encodeurs, il s'agit de n'importe quels vecteurs de dimension réduite provenant d'un réseau profond.

Ces vecteurs sont très utiles car ils sont de petites dimensions mais sont porteurs d'information sémantique. Autrement dit, ils "encodent" les données d'entrée du réseau, et sont un condensé des informations importantes.

> Prenons un exemple.
> Vous entrainez un réseau à reconnaitre des voitures (donc supervisé). Une fois que votre réseau est efficace sur des voitures, vous le coupez quelques couches avant la fin. Vous obtenez un réseau qui prend en entrée des images et donne en sortie un vecteur de petite dimension, disons 64. Comme votre réseau aura appris à extraire les informations utiles pour reconnaitre une voiture (roues, forme, etc), ces vecteurs comporteront toutes les informations nécessaires à reconnaitre des voitures.
Ensuite, si vous donnez à ce réseau des images de motos et de camions, une simple comparaison des vecteurs de sortie pourra indiquer si il s'agit d'une moto ou d'un camion. Il sera évidemment toujours peu efficace pour reconnaitre des objets éloignés des voitures, comme des visages par exemple. 

### But de cette dernière partie

Le but général de cette dernière partie est de vous montrer quelques outils classiques de machine learning et d'analyse des données. Dans la continuité de la première partie, l'idée est de pouvoir analyser les vecteurs latents provenants de l'auto-encoder. En particulier, vous serez capables d'appliquer un clustering non supervisé sur l'espace latent pour pouvoir retrouver les 10 classes de CIFAR-10, sans jamais utiliser les labels.

Pour cela on va voir chacun de ces outils de machine learning en s'entrainant sur la dataset "Wine" qui contient 3 classes de différents vins, dont chacun a 13 attributs (principalement les taux de présence de composés chimiques).

Nous verrons :
- l'importance de la normalisation des données
- la visualisation par t-SNE
- la réduction de dimensions par ACP
- le clustering par K-means
- le clustering par agglomération

Ensuite, vous pourrez appliquer un clustering aux données de l'auto-encodeur. Celui qui aura le meilleur score gagne le TP.

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from sklearn.datasets import load_wine
data = load_wine()
train_data = data['data']
train_labels = data['target']

print(f"Les {train_data.shape[0]} vecteurs sont de dimension {train_data.shape[1]}.")

### Analyse des données

Il est souvent nécessaire de réduire la dimension des données que l'on manipule. Un encodeur (par exemple la première partie d'un auto-encodeur) est déjà un moyen de réduire les dimensions en codant des images dans un espace vectoriel beaucoup plus petit.
Un auto-encodeur a le rôle très particulier d'encoder des images (ou autres données structurées) mais d'autres algorithmes s'appliquent à des données vectorielles classiques.

La réduction de dimension permet à la fois à mieux visualiser les données (par exemple on peut réduire les vecteurs à 2 ou 3 dimensions pour afficher les données sur un graphique), et à la fois de pré-traiter les données. Pré-traiter les données en réduisant le nombre de dimensions est parfois indispensable, en effet, certains algorithmes - en particulier ceux basés sur la distances- supportent mal les très hautes dimensions.
> Pour ceux qui ca intéresse, voir "malédiction de la dimension". [version simple (wiki)](<https://fr.wikipedia.org/wiki/Fl%C3%A9au_de_la_dimension>) / [version hardcore](<https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.64.2646&rep=rep1&type=pdf>).

Dans la suite du TP, nous verrons la transformation t-SNE pour la visualisation, et l'ACP pour le pre-process.

**Il est important de noter que les données que l'on utilise pour le TP n'ont que 13 dimensions. C'est minuscule et ça ne nécessite pas de réduire encore plus la dimension, l'ACP est là à titre pédagogique. Par contre, pour la dernière question du TP, l'ACP sera très utile.**

#### Visualisation (t-SNE)

Le t-SNE (pour t-distributed stochastic neighbor embedding) est un algo de réduction de dimension spécialement conçu pour la visualisation de données. L'algorithme tend à maintenir les proximités entre les points depuis l'espace de grande dimension vers l'espace de petite dimension, selon une métrique donnée.

> Si vous voulez creuser son fonctionnement (ça demande quelques connaissances en théorie de l'information) : 
> [version simple](<https://aiaspirant.com/introduction-to-t-distributed-stochastic-neighbor-embeddingt-sne/>) / [version complète](<https://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf>).

Notez que ca a été inventé par **Geoffrey Hinton**, un monument de l'IA, père de beaucoup d'autres méthodes. Vous allez probablement recroiser son nom souvent.

Comme on va le voir, le t-SNE n'est pas déterministe (c'est-à-dire qu'il a une part d'aléatoire dans son exécution), et c'est une chose à ne pas oublier quand vous l'utiliserez.

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
t0 = time()
tsne_results = tsne.fit_transform(train_data)
print(f"Done in {time() - t0}s")
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
plt.scatter(tsne_results[:,0], tsne_results[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Transformation t-SNE")

Le t-SNE n'est pas déterministe. Vous pouvez le vérifier en lançant plusieurs fois la réduction, vous n'aurez jamais le même résultat.

#### Normalisation des données

Beaucoup d'algorithmes se basent sur les distances entre les données. Or la qualité des calculs des distances dépend fortement de la normalisation des axes. Par exemple, un axe qui a des valeurs entre 0 et 100 aura une influence beaucoup plus forte car ses valeurs sont beaucoup plus grandes que celles prises dans un axe ayant des valeurs entre 0 et 1, et on verrait une distorsion artificielle le long de l'axe non normalisé.

Il y a principalement deux façons de normaliser, soit on ramène toutes les données entre un min et un max (MinMaxScaler), soit on normalise les données pour que chaque axe soit centré et réduit (StandardScaler).

In [ ]:
for axe in range(train_data.shape[1]):
  extract = train_data[:, [axe]]
  print(f"axe {axe} - min {min(extract)} - max {max(extract)}")

Les différentes composantes des vecteurs d'entrainement n'ont pas du tout les mêmes plages de valeurs. Pour vous entrainer à manipuler les transformateurs de normalisation, affichez en dessous trois transformations t-SNE, une avec les données brutes, une avec une normalisation MinMax et une avec une normalisation gaussienne.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler



tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_raw = tsne.fit_transform(train_data)

scaler = StandardScaler()
train_data_norm = scaler.fit_transform(train_data)

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_norm = tsne.fit_transform(train_data_norm)

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_minmax = scaler.fit_transform(train_data)

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_minmax = tsne.fit_transform(train_data_minmax)



colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
plt.figure(figsize=(16,7))
ax1 = plt.subplot(1, 3, 1)
plt.scatter(tsne_results_raw[:,0], tsne_results_raw[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("données non normées")
ax2 = plt.subplot(1, 3, 2)
plt.scatter(tsne_results_norm[:,0], tsne_results_norm[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("données normées minmax")
ax3 = plt.subplot(1, 3, 3)
plt.scatter(tsne_results_minmax[:,0], tsne_results_minmax[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("données normées standard")

#### Réduction de dimension (ACP/PCA)

L'Analyse en Composante Principale est une autre méthode de réduction de dimension. L'ACP repose sur un principe intuitif : on recherche les directions dans lesquelles les données s'étalent le plus.

On vous encourage à regarder l'idée générale sur les liens ci-contre, ca se comprend très vite. 
[version simple (wiki)](<https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales>) / [version complète](<https://www.researchgate.net/publication/309165405_Principal_component_analysis_-_a_tutorial>).

C'est la technique qui est de très loin la plus utilisée pour la réduction de dimension. Elle permet notamment de savoir combien d'information on perd lorsque l'on réduit la dimension. Elle a beaucoup d'autres avantages qui seront détaillés dans la suite.

In [ ]:
from sklearn.decomposition import PCA
n_components = 10
pca = PCA(n_components=n_components, svd_solver='full', whiten=True)

t0 = time()
pca_results = pca.fit_transform(train_data_norm)
print(f"Done in {time() - t0}s")

components = range(1, n_components+1)
explained_variance = pca.explained_variance_ratio_
plt.show()
plt.figure(figsize=(16,7))
ax1 = plt.subplot(1, 2, 1)
plt.plot(components, explained_variance)
plt.xlabel("composantes")
plt.ylabel("proportion de variance expliquée")
plt.title("Variance expliquée par composante")
ax2 = plt.subplot(1, 2, 2)
plt.plot(components, np.cumsum(explained_variance))
print('\nExplained variation per principal component: \n {} \n'.format(pca.explained_variance_ratio_))
plt.xlabel("composantes")
plt.ylabel("proportion de variance expliquée cumulée")
plt.title("Variance expliquée cumulée")

Le graphe précédent montre la proportion de variance expliquée sur chacune des composantes principales. Intuitivement, il s'agit de la proportion d'information contenue dans chacune des composantes.
Suivant vos besoins, vous pouvez donc garder les composantes principales qui vous sont utiles.
En ce qui concerne les données que l'on manipule ici, on remarque que 6 dimensions sont nécessaires pour expliquer plus de 90% de la variance.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2, svd_solver='full', whiten=True)

t0 = time()
pca_results = pca.fit_transform(train_data_norm)
print(f"Done in {time() - t0}s")

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
plt.scatter(pca_results[:,0], pca_results[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Les deux premières composantes de l'ACP")

#### Pourquoi une ACP, pourquoi un t-SNE ?

On l'a vu, le t-SNE a l'air d'être efficace pour réduire les dimensions, alors pourquoi on utilise une ACP pour le traitement réel des données ? 

C'est parce-que la transformation t-SNE souffre de quelques points noirs :

- Le t-SNE, comme on l'a vu, n'est pas déterministe. La sortie change à chaque exécution, ce qui peut poser problème pour la manipulation des données après la transformation. L'ACP par contre fournit des résultats déterministes.

- Le t-SNE se base sur les relations entre les points voisins mais ne permet pas toujours de visualiser les tendances globales. Vous aurez toujours des données bien étalées dans l'espace alors que les données d'entrée ne le sont pas forcément. C'est aussi ce qui rend le t-SNE parfois très utile pour visualiser les données.

- Les composantes principales en sortie d'ACP portent en elles-mêmes une signification puisqu'elles se décomposent selon des composantes d'entrée. Si les données d'entrée ont un sens, alors les sorties auront un sens.
> Par exemple, si on veut prédire le prix d'une maison, une ACP nous donne des informations du type : "La composante principale qui explique le prix d'une maison est composée à 20% de sa localisation, à 30% de sa surface, ...".

- L'ACP permet d'obtenir les matrices de passage d'un espace à l'autre qui peuvent être utilisées pour projeter facilement de nouvelles données dans l'espace réduit.

- L'ACP sert de base a beaucoup d'autres algorithmes, qui sont utiles pour des cas particuliers ou des cas extremes (peu de données, données mal réparties, etc).

In [ ]:
from sklearn.datasets import load_breast_cancer
data_cancer = load_breast_cancer()
train_data_cancer = data_cancer['data']
train_labels_cancer = data_cancer['target']

scaler_cancer = StandardScaler()
train_data_cancer = scaler.fit_transform(train_data_cancer)

pca_cancer = PCA(n_components=2, svd_solver='full', whiten=True)
pca_results_cancer = pca.fit_transform(train_data_cancer)

tsne_cancer = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results_cancer = tsne.fit_transform(train_data_cancer)

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
plt.figure(figsize=(16,7))
ax1 = plt.subplot(1, 2, 1)
plt.title("Transformation t-SNE")
plt.scatter(tsne_results_cancer[:,0], tsne_results_cancer[:,1], c=train_labels_cancer, cmap=matplotlib.colors.ListedColormap(colors))
ax2 = plt.subplot(1, 2, 2)
plt.scatter(pca_results_cancer[:,0], pca_results_cancer[:,1], c=train_labels_cancer, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Transformation par ACP")

En comparant les deux algorithmes, on remarque l'étalement provoqué par le t-SNE. C'est utile pour bien visualiser les données mais on perd l'information qui concerne les tendances globales des données. L'ACP donne des nuages de points plus denses ce qui les rend moins simple à visualiser, mais la tendance globale du nuage de point est bien visible.

#### Quelques autres algos

D'autres algorithmes permettent la réduction de dimension dans des cas particuliers :

- La PLS (ou régression des moindres carrés partiels) est équivalente à l'ACP mais elle est supervisée. Ça permet de chercher les composantes principales qui permettent le mieux de distinguer les classes (qui sont donc connues).

- L'algorithme FCA analyse les relations entre deux variable qualitatives. C'est donc une ACP pour des données d'entrée non numériques, et pour lesquelles on ne peut pas facilement définir de distance.

### Classification non supervisée (clustering)

#### K-means

Le K-means est un algorithme de clustering. Il fonctionne donc de manière non supervisé et se base sur une distance pour comparer les vecteurs.

> Le K-means est un algo itératif. À l'initialisation, il prend k points répartis aléatoirement (que nous appelerons les *centroïdes*). Chaque itération comporte deux étapes :
- On attribue chaque point des données à un des *centroïdes*. Dans sa forme basique, chaque donnée est attribuée au *centroïde* qui est le plus proche.
- Pour chaque *centroïde*, on regroupe tous les points qui lui sont attribués, ceux-ci forment un cluster ; puis chaque *centroïde* est mis à jour comme étant le barycentre de son nouveau cluster.

>  Ces étapes sont répétées jusqu'à l'arrêt par un critère de convergence.

<img src="https://stanford.edu/~cpiech/cs221/img/kmeansViz.png" width="440" height="260" align="center"/>

Pour avoir plus d'infos sur le K-means : [Anas Al-Masri : How does K-means work ?](<https://towardsdatascience.com/how-does-k-means-clustering-in-machine-learning-work-fdaaaf5acfa0>)


Il a l'avantage d'être simple et rapide, mais nécessite de donner le nombre de clusters k, et c'est une information que l'on n'a pas toujours. De plus, le choix des valeurs initiales des *centroïdes* influent beaucoup sur le résultat.

La solution classique pour pallier ce problème consiste à initialiser les *centroïdes* aléatoirement et lancer l'algorithme plusieurs fois, et avec différentes valeurs de k. Il existe aussi une variante kmeans++ qui intialise les *centroïdes* de manière à optimiser le résultat.



In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0)
prediction = kmeans.fit_predict(train_data_norm)

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

plt.figure(figsize=(16,7))
ax1 = plt.subplot(1, 2, 1)
plt.scatter(pca_results[:,0], pca_results[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Données annotées")
ax2 = plt.subplot(1, 2, 2)
plt.scatter(pca_results[:,0], pca_results[:,1], c=prediction, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Predictions du K-means")

#### Clustering agglomératif

Le clustering agglomeratif consiste à constuire un arbre de similitude entre les données.
Dans sa forme la plus simple, il s'agit de réunir itérativement les données les plus proches.

> En deux mots. On initialise en considèrant chacun des points comme un cluster ayant un unique point. Ensuite à chaque étape on calcule les barycentres de chacun des clusters, et on regroupe les clusters dont les barycentres sont les plus proches.

<img src="https://cedric.cnam.fr/vertigo/Cours/RCP216/_images/hierarchicalClustering.png" width="440" height="160" align="center"/>

Pour en savoir plus : 
[version simple (wiki)](<https://fr.wikipedia.org/wiki/Regroupement_hi%C3%A9rarchique>) / [version complète](<lien>)

Le grand avantage de cette technique c'est qu'elle ne nécessite pas de connaitre k en avance. On peut lancer l'algorithme et couper l'arbre au nombre de cluster qui nous convient ou qui donne le meilleurs score.

Il existe différentes facon de regrouper les clusters, et différentes variantes. Par exemple, on peut aussi considérer tous nos points comme un seul grand cluster, puis le diviser itérativement de manière optimale. 

In [ ]:
from sklearn.cluster import AgglomerativeClustering

AC = AgglomerativeClustering()
AC.set_params(n_clusters=3)
prediction = AC.fit_predict(train_data_norm)

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

plt.figure(figsize=(16,7))
ax1 = plt.subplot(1, 2, 1)
plt.scatter(pca_results[:,0], pca_results[:,1], c=train_labels, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Données annotées")
ax2 = plt.subplot(1, 2, 2)
plt.scatter(pca_results[:,0], pca_results[:,1], c=prediction, cmap=matplotlib.colors.ListedColormap(colors))
plt.title("Predictions de l'AC")

La bbilothèque Scikit ne permet pas de visualiser le clustering hiérarchique. Ci-dessous on utilise Scipy pour afficher le dendogramme. On voit qu'il est assez facile de couper l'arbre à la bonne hauteur pour choisir le nombre de clusters.

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram

hc_complete = linkage(train_data_norm, "complete")

plt.figure(figsize=(25, 10))
plt.title('Dendrogram')
plt.xlabel('donnée')
plt.ylabel('distance')
dendrogram(
    hc_complete,
    leaf_rotation=90.,
    leaf_font_size=8.,
)
plt.show()

#### Score NMI

Trouver une bonne métrique pour mesurer la qualité d'un clustering non supervisé n'est pas si simple. En effet, la plupart des métriques classiques se contentent de comparer les *labels* prédits aux vrais labels. Mais dans notre cas, nous avons deux problèmes :

  - En non-supervisé sur des données brutes, on n'a pas forcément ces *labels* vrais. Il faut donc se contenter de mesurer à quel point les classes sont denses (**inertie intra-cluster**), et à quel point les différentes classes sont éloignées (**inertie inter-cluster**). Il existe beaucoup de métriques de ce type, on peut citer le **coefficient de Silhouette** ou encore **l'indice de Calinski-Harabasz.**

- Lorsque l'on a les labels vrais, ce n'est pas gagné non plus. En effet, si l'algorithme prédit [1, 1, 2, 2, 3, 3] alors que les vraies valeurs sont [3, 3, 1, 1, 2, 2], l'algorithme n'a pas fait d'erreur de clustering, mais une comparaison label à label donnerait un score nul. Il faut donc une métrique qui ne soit pas basée sur le nom des classes. On peut utiliser **l'Adjusted Rand Index (ARI)** ou encore **l'indice de Jacard**, mais pour le TP je vous propose le score **NMI (Normalized Mutual Information)**.



> Pour plus d'informations sur les mesures de qualité du clustering, voir 
[Manimaran : Clustering Evaluation strategies](<https://towardsdatascience.com/clustering-evaluation-strategies-98a4006fcfc>)

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI


kmeans = KMeans(n_clusters=3, random_state=0)
prediction = kmeans.fit_predict(train_data_norm)

print("Score NMI :", NMI(prediction, train_labels))

#À vous de jouer ! Tracez le score en MNI en fonction du nombre de cluster (de 5 à 20) :

cluster = []
scores = []

for k in range(1, 10):
  kmeans = KMeans(n_clusters=k, random_state=0)
  prediction = kmeans.fit_predict(train_data_norm)
  cluster.append(k)
  score = NMI(prediction, train_labels)
  scores.append(score)
  print(f"\rk = {k}", end='')

plt.plot(cluster, scores)
plt.xlabel("nombre de clusters")
plt.ylabel("score NMI")
plt.title("Qualité du clustering en fonction du nombre de clusters")

On voit que le meilleur clustering se fait pour k=3. Il n'y a rien de très étonnant puisque nous avions trois classes dans les données. Lorsque vous manipulez des données sans connaitre k, cette méthode est un bon moyen de savoir quel est le nombre optimal de classes.

### Pipeline sci-kit
(cette sous-partie présente juste une astuce de programmation, peu connue, et très utile)

La manière la plus propre d'utiliser sci-kit lorsque l'on a besoin d'un enchainement de plusieurs transformations, c'est d'utiliser les **pipelines**.

Il s'agit d'un regroupement de plusieurs objets *transformers* réunis dans un seul objet ```Pipeline()```.

```
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('nom_1', objet_transformateur_1()),
    ('nom_2', objet_transformateur_2()),
    etc...
])
```

Pour manipuler votre Pipeline, vous pouvez utiliser les même méthodes que pour les transformateurs : ```.fit()```, ```.predict()```, ```.fit_predict()```, etc.

> Vous pouvez aussi réécrire (en subclassing) les classes ```BaseEstimator```, ```TransformerMixin``` pour faire vos propres transformateurs et les inclure dans des pipelines.

Pour vous entrainer, essayez de  créer et entrainer une pipeline qui regroupe une ACP et un Kmeans sur les données ```train_data_viz```.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

#ceci est un exemple de pipeline, mais en fait il est inutile de faire une ACP ici
pipeline = Pipeline([
    ('pca', PCA(n_components=10, svd_solver='full', whiten=True)),
    ('kmeans', KMeans(n_clusters=3, random_state=0)),
])


pipeline.fit(train_data_norm)
prediction = pipeline.predict(train_data_norm)

print("Score NMI :", NMI(prediction, train_labels))

# À vous de jouer en utilisant tout ça !

Pour finir le TP, essayez de faire une classification non-supervisée sur CIFAR-10. Celui qui a le score NMI le plus haut gagne le TP.

> Indice :
> Vous pouvez extraire des caractéristiques grace à un auto-encodeur, puis les classifier avec un algorithme de clustering de votre choix.

N'oubliez pas que vous ne devrez pas utiliser les *labels*, sauf pour le calcul du score NMI.

---
PS : Si vous êtes motivés, vous pouvez coder un k-means *from scratch* (avec numpy), c'est pas hyper long et ca permet de bien comprendre l'algo.